In [137]:
import os
import glob
import json
import struct
from datetime import datetime, timezone
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
pd.set_option('display.max_rows', None)

directory_path_original = './logs/dut0_original/logs_client'
directory_path_ecc_bypass = './logs/dut0_ecc_bypass/logs_client'
directory_path_iv_tee = './logs/dut0_iv_tee/logs_client'
file_pattern = 'Client_dut*'

# Use glob to find all files matching the pattern
file_paths_original = glob.glob(os.path.join(directory_path_original, file_pattern))
file_paths_ecc_bypass = glob.glob(os.path.join(directory_path_ecc_bypass, file_pattern))
file_paths_iv_tee = glob.glob(os.path.join(directory_path_iv_tee, file_pattern))

In [138]:
crcTable = [    
    0x0000,
    0x1021,
    0x2042,
    0x3063,
    0x4084,
    0x50A5,
    0x60C6,
    0x70E7,
    0x8108,
    0x9129,
    0xA14A,
    0xB16B,
    0xC18C,
    0xD1AD,
    0xE1CE,
    0xF1EF,
    0x1231,
    0x0210,
    0x3273,
    0x2252,
    0x52B5,
    0x4294,
    0x72F7,
    0x62D6,
    0x9339,
    0x8318,
    0xB37B,
    0xA35A,
    0xD3BD,
    0xC39C,
    0xF3FF,
    0xE3DE,
    0x2462,
    0x3443,
    0x0420,
    0x1401,
    0x64E6,
    0x74C7,
    0x44A4,
    0x5485,
    0xA56A,
    0xB54B,
    0x8528,
    0x9509,
    0xE5EE,
    0xF5CF,
    0xC5AC,
    0xD58D,
    0x3653,
    0x2672,
    0x1611,
    0x0630,
    0x76D7,
    0x66F6,
    0x5695,
    0x46B4,
    0xB75B,
    0xA77A,
    0x9719,
    0x8738,
    0xF7DF,
    0xE7FE,
    0xD79D,
    0xC7BC,
    0x48C4,
    0x58E5,
    0x6886,
    0x78A7,
    0x0840,
    0x1861,
    0x2802,
    0x3823,
    0xC9CC,
    0xD9ED,
    0xE98E,
    0xF9AF,
    0x8948,
    0x9969,
    0xA90A,
    0xB92B,
    0x5AF5,
    0x4AD4,
    0x7AB7,
    0x6A96,
    0x1A71,
    0x0A50,
    0x3A33,
    0x2A12,
    0xDBFD,
    0xCBDC,
    0xFBBF,
    0xEB9E,
    0x9B79,
    0x8B58,
    0xBB3B,
    0xAB1A,
    0x6CA6,
    0x7C87,
    0x4CE4,
    0x5CC5,
    0x2C22,
    0x3C03,
    0x0C60,
    0x1C41,
    0xEDAE,
    0xFD8F,
    0xCDEC,
    0xDDCD,
    0xAD2A,
    0xBD0B,
    0x8D68,
    0x9D49,
    0x7E97,
    0x6EB6,
    0x5ED5,
    0x4EF4,
    0x3E13,
    0x2E32,
    0x1E51,
    0x0E70,
    0xFF9F,
    0xEFBE,
    0xDFDD,
    0xCFFC,
    0xBF1B,
    0xAF3A,
    0x9F59,
    0x8F78,
    0x9188,
    0x81A9,
    0xB1CA,
    0xA1EB,
    0xD10C,
    0xC12D,
    0xF14E,
    0xE16F,
    0x1080,
    0x00A1,
    0x30C2,
    0x20E3,
    0x5004,
    0x4025,
    0x7046,
    0x6067,
    0x83B9,
    0x9398,
    0xA3FB,
    0xB3DA,
    0xC33D,
    0xD31C,
    0xE37F,
    0xF35E,
    0x02B1,
    0x1290,
    0x22F3,
    0x32D2,
    0x4235,
    0x5214,
    0x6277,
    0x7256,
    0xB5EA,
    0xA5CB,
    0x95A8,
    0x8589,
    0xF56E,
    0xE54F,
    0xD52C,
    0xC50D,
    0x34E2,
    0x24C3,
    0x14A0,
    0x0481,
    0x7466,
    0x6447,
    0x5424,
    0x4405,
    0xA7DB,
    0xB7FA,
    0x8799,
    0x97B8,
    0xE75F,
    0xF77E,
    0xC71D,
    0xD73C,
    0x26D3,
    0x36F2,
    0x0691,
    0x16B0,
    0x6657,
    0x7676,
    0x4615,
    0x5634,
    0xD94C,
    0xC96D,
    0xF90E,
    0xE92F,
    0x99C8,
    0x89E9,
    0xB98A,
    0xA9AB,
    0x5844,
    0x4865,
    0x7806,
    0x6827,
    0x18C0,
    0x08E1,
    0x3882,
    0x28A3,
    0xCB7D,
    0xDB5C,
    0xEB3F,
    0xFB1E,
    0x8BF9,
    0x9BD8,
    0xABBB,
    0xBB9A,
    0x4A75,
    0x5A54,
    0x6A37,
    0x7A16,
    0x0AF1,
    0x1AD0,
    0x2AB3,
    0x3A92,
    0xFD2E,
    0xED0F,
    0xDD6C,
    0xCD4D,
    0xBDAA,
    0xAD8B,
    0x9DE8,
    0x8DC9,
    0x7C26,
    0x6C07,
    0x5C64,
    0x4C45,
    0x3CA2,
    0x2C83,
    0x1CE0,
    0x0CC1,
    0xEF1F,
    0xFF3E,
    0xCF5D,
    0xDF7C,
    0xAF9B,
    0xBFBA,
    0x8FD9,
    0x9FF8,
    0x6E17,
    0x7E36,
    0x4E55,
    0x5E74,
    0x2E93,
    0x3EB2,
    0x0ED1,
    0x1EF0,
]


Recursively checking all the log files and sorting on Unix timestamp

In [139]:
def parse_files(file_paths):
    frames = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            for json_line in file:
                # Parse frame
                try:
                    frame = json.loads(json_line)
                except Exception as error:
                    print(f"Failed decoding JSON for file {file_path}: {error}")
                frames.append(frame)
    return frames

Constructing frames

In [140]:
frames_original = sorted(parse_files(file_paths_original), key=lambda x: x["timestamp"])
frames_ecc_bypass = sorted(parse_files(file_paths_ecc_bypass), key=lambda x: x["timestamp"])
frames_iv_tee = sorted(parse_files(file_paths_iv_tee), key=lambda x: x["timestamp"])

df_original = pd.DataFrame(frames_original)
df_ecc_bypass = pd.DataFrame(frames_ecc_bypass)
df_iv_tee = pd.DataFrame(frames_iv_tee)


Frame helper functions

In [141]:
def decode_frame(frame_bytes):
    # Desconstructing the frame
    header = frame_bytes[0]
    frame_id = frame_bytes[1]
    payload_length = frame_bytes[2]
    payload = frame_bytes[3 : 3 + payload_length]
    crc_bytes = frame_bytes[3 + payload_length : 5 + payload_length]
    tail = frame_bytes[-1]

    # Concatenating the CRC bytes into a single number
    # Assuming CRC is in big-endian format
    crc = (crc_bytes[0] << 8) | crc_bytes[1]

    # Converting payload to hex representation
    payload_hex = [hex(byte) for byte in payload]

    crc_check = check_crc(payload, payload_length, crc)

    data = None

    if crc_check is False:
        print(f"CRC Check failed!")
    else:
        data = parse_payload(payload, frame_id)

    return data

In [142]:
def check_crc(payload, payload_length, crc_value):
        INITIAL_REMAINDER = 0xFFFF
        FINAL_XOR_VALUE = 0x0000
        remainder = INITIAL_REMAINDER

        for byte in range(payload_length):
            data = payload[byte] ^ (remainder >> (16 - 8))
            remainder = crcTable[data] ^ (remainder << 8) & 0xFFFF

        return crc_value == (remainder ^ FINAL_XOR_VALUE)

In [143]:
# +--------+--------------------+-------------+---------------+
# | Format | C Type             | Python type | Standard size |
# +--------+--------------------+-------------+---------------+
# | x      | pad byte           | no value    | (7)           |
# | c      | char               | bytes       | 1             |
# | b      | signed char        | integer     | 1             |
# | B      | unsigned char      | integer     | 1             |
# | ?      | _Bool              | bool        | 1             |
# | h      | short              | integer     | 2             |
# | H      | unsigned short     | integer     | 2             |
# | i      | int                | integer     | 4             |
# | I      | unsigned int       | integer     | 4             |
# | l      | long               | integer     | 4             |
# | L      | unsigned long      | integer     | 4             |
# | q      | long long          | integer     | 8             |
# | Q      | unsigned long long | integer     | 8             |
# | n      | ssize_t            | integer     | (3)           |
# | N      | size_t             | integer     | (3)           |
# | e      | (6)                | float       | 2             |
# | f      | float              | float       | 4             |
# | d      | double             | float       | 8             |
# | s      | char[]             | bytes       | (9)           |
# | p      | char[]             | bytes       | (8)           |
# | P      | void*              | integer     | (5)           |
# +--------+--------------------+-------------+---------------+


frame_id_formatting = {
    "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB": 0,  # test frame
    # "II": 1,  # fini
    "IIIIIIIIIIIIIII": 1,  # fini
    "IIIIII": 16,  # exception
}


def parse_payload(data, frame_id):
    # Ensure that the data is a bytes object
    # if not isinstance(data, bytes):
    #     raise ValueError("Data must be bytes")

    # Find the format string for the given frame_id
    format_str = None
    unpacked_data = ()
    for fmt_str, id in frame_id_formatting.items():
        if id == frame_id:
            format_str = fmt_str
            break

    if format_str is None:
        raise ValueError(f"No format string found for frame ID {frame_id}")

    # Unpack the data dynamically based on the format string
    try:
        unpacked_data = struct.unpack(format_str, data)
    except struct.error as e:
        print(f"Error unpacking data with format {format_str}: {e}, data {data}")

    # # Convert bytes to string for field4 if necessary
    # field4 = field4.decode("utf-8").rstrip("\x00")

    # for field in unpacked_data:
    #     print(hex(field))

    return unpacked_data


Converting Unix timestamp to UTC

In [144]:
# def convert_unix_to_utc(pd):
#     crc_errors = 0
#     frame_parsing_errors = 0
#     for frame in frames:
#         frame['timestamp'] = str(datetime.utcfromtimestamp(float(frame['timestamp'])).replace(tzinfo=timezone.utc).strftime('%Y-%m-%d %H:%M:%S.%f UTC'))
#         if 'data' in frame:
#             try:
#                 decoded_frame = decode_frame(bytes.fromhex(frame['data']))
#                 #CRC check failed
#                 if decoded_frame == None:
#                     converted_frames.append({'timestamp': frame['timestamp'], 'error': "CRC check error"})
#                     crc_errors += 1
#                 # Exception
#                 elif len(decoded_frame) < 15:
#                     converted_frames.append({'timestamp': frame['timestamp'], 'error': f"Frame parsing error. Data field too small: {decode_frame}"})
#                 # Data correct
#                 else:
#                     frame['data'] = decoded_frame
#                     converted_frames.append(frame)
#             except Exception as error:
#                 frame_parsing_errors += 1
#                 converted_frames.append({'timestamp': frame['timestamp'], 'error': "Frame parsing error (possibly due to comm failure)"})
#                 #print(f"Parsing error! {error}")
#         elif 'event' in frame:
#             converted_frames.append(frame)
#     return converted_frames, crc_errors, frame_parsing_errors

df_original['timestamp'] = pd.to_datetime(df_original['timestamp'], unit='s').dt.tz_localize('UTC')
df_ecc_bypass['timestamp'] = pd.to_datetime(df_ecc_bypass['timestamp'], unit='s').dt.tz_localize('UTC')
df_iv_tee['timestamp'] = pd.to_datetime(df_iv_tee['timestamp'], unit='s').dt.tz_localize('UTC')

Filtering out duplicate entries

In [145]:
df_original = df_original.drop_duplicates()
df_ecc_bypass = df_ecc_bypass.drop_duplicates()
df_iv_tee = df_iv_tee.drop_duplicates()

Parsing the fields based on frame type

In [146]:
# data_frame = []

# for frame in unique_frames:
#     if 'data' in frame:
#         parsed_payload = (frame['timestamp'],) + frame['data']
#         keys = ['timestamp', 'total_errors', 'mcycle', 'minstret', 'imem_se', 'imem_de', 'dmem_se', 'dmem_de', 'regfile_se', 'regfile_de', 'iv', 'jump', 'branch', 'dsp_t', 'trap', 'illegal']
#         mapping = {num: key for num, key in zip(keys, parsed_payload)}
#         data_frame.append(mapping)

# df = pd.DataFrame(data_frame)

def data_dict(data_hex):
    try:
        data_tup = decode_frame(bytes.fromhex(data_hex))
        return data_tup
    except Exception as error:
        print(f"Parsing error! {error}")
        return data_hex 

def parse_data(df):
    # Apply the function only to rows where 'Column2' is not empty
    mask = df['data'].notnull() & (df['data'] != '')

    df.loc[mask, 'data'] = df.loc[mask, 'data'].apply(data_dict)

parse_data(df_original)
df_original


Parsing error! non-hexadecimal number found in fromhex() arg at position 0
Parsing error! non-hexadecimal number found in fromhex() arg at position 0
Parsing error! non-hexadecimal number found in fromhex() arg at position 0
Parsing error! non-hexadecimal number found in fromhex() arg at position 0
Parsing error! non-hexadecimal number found in fromhex() arg at position 0
Parsing error! non-hexadecimal number found in fromhex() arg at position 0
Parsing error! non-hexadecimal number found in fromhex() arg at position 0
Parsing error! non-hexadecimal number found in fromhex() arg at position 0


,message,type,id,timestamp,data,event
0,,Serial dut0_monitor,20,2023-12-06 17:40:11.329293056+00:00,"(0, 284115938, 1782107734, 0, 0, 0, 0, 0, 0, 0...",NaN
1,,Serial dut0_monitor,20,2023-12-06 17:40:17.023052032+00:00,"(0, 284115925, 1815731912, 0, 0, 0, 0, 0, 0, 0...",NaN
2,,Serial dut0_monitor,20,2023-12-06 17:40:22.718929664+00:00,"(0, 284115934, 1849356090, 0, 0, 0, 0, 0, 0, 0...",NaN
3,,Serial dut0_monitor,20,2023-12-06 17:40:28.420879360+00:00,"(0, 284115920, 1882980268, 0, 0, 0, 0, 0, 0, 0...",NaN
4,,Serial dut0_monitor,20,2023-12-06 17:40:34.115907840+00:00,"(0, 284115935, 1916604446, 0, 0, 0, 0, 0, 0, 0...",NaN
5,,Serial dut0_monitor,20,2023-12-06 17:40:39.810676992+00:00,"(0, 284174329, 1950234149, 0, 0, 0, 0, 0, 0, 0...",NaN
6,,Serial dut0_monitor,20,2023-12-06 17:40:45.513253376+00:00,"(0, 284115919, 1983858327, 0, 0, 0, 0, 0, 0, 0...",NaN
7,,Serial dut0_monitor,20,2023-12-06 17:40:51.207973376+00:00,"(0, 284115921, 2017482505, 0, 0, 0, 0, 0, 0, 0...",NaN
8,,Serial dut0_monitor,20,2023-12-06 17:40:56.902738176+00:00,"(0, 284115929, 2051106683, 0, 0, 0, 0, 0, 0, 0...",NaN
9,,Serial dut0_monitor,20,2023-12-06 17:41:02.598188032+00:00,"(0, 284115936, 2084730861, 0, 0, 0, 0, 0, 0, 0...",NaN


# DSP timeouts

In [147]:
filtered = df[df['dsp_t'] > 0]
filtered

NameError: name 'df' is not defined

# IV detections

In [ ]:
filtered = df[df['iv'] > 0]
filtered

# IV FPR higher than 2%

In [ ]:
# scaler = MinMaxScaler()
# df['imem_se'] = scaler.fit_transform(df[['imem_se']])
# df['iv'] = scaler.fit_transform(df[['iv']])
# df_no_duplicates = df.drop_duplicates(subset='iv', keep='first')
# fpr_violation_condition = df_no_duplicates['iv'] < (0.98*df_no_duplicates['imem_se'])
# fpr_violation_condition

# Double bit errors

In [ ]:
df_no_duplicates = df.drop_duplicates(subset='imem_de', keep='first')
df_no_duplicates

In [ ]:
fpr_violation_condition = df['iv'] > (0.98*df['imem_se'])
# df[fpr_violation_condition]

# Filtering based on SmartFusion2 reset. Excluding DMEM and Register file because of scrubbing and overwriting

In [ ]:
mask = (df['imem_se'] < df['imem_se'].shift()-10) | (df['imem_de'] < df['imem_de'].shift()-10)
resets = df[mask | mask.shift(-1)]
pd.concat([df.head(1), resets])

# Multiple explanations for differences between IMEM and instruction validator detections
## Significant less detections by IV compared to IMEM

## Significant more detections by IV compared to IMEM